In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.18.0 requires torch

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from peft import LoraConfig, get_peft_model, TaskType
import time

In [3]:
#Load dataset
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [5]:
dash_line = '-'.join('' for x in range(100))

In [50]:
#Prompt engineering - zero shot inferencing
example_indices = [1,300,450]
for i, index in enumerate(example_indices):
  dialogue = dataset['train'][index]['dialogue']
  summary = dataset['train'][index]['summary']
  prompt = f"""
  {dialogue}
  Summary:
  """
  inputs = tokenizer(prompt, return_tensors='pt')
  model_output = original_model.generate(inputs["input_ids"], max_new_tokens=50)[0]
  output = tokenizer.decode(model_output,skip_special_tokens=True)
  print(f'INPUT PROMPT:\n{prompt}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
  print(dash_line)
  print(f'MODEL GENERATION - ZERO SHOT:\n{output}')


INPUT PROMPT:

  #Person1#: Hello Mrs. Parker, how have you been?
#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.
#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.
#Person2#: What about Rubella and Mumps?
#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.
#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!
#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little.
  Summary:
  
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the recor

In [54]:
#One-shot inferencing
example_index = 100
dialogue = dataset['train'][example_index]['dialogue']
summary = dataset['train'][example_index]['summary']
prompt = f"""
Dialogue:

{dialogue}

Summary:
{summary}
"""

example_index_to_summarize = 300
dialogue_to_summarize = dataset['train'][example_index_to_summarize]['dialogue']
summarized_dialogue = dataset['train'][example_index_to_summarize]['summary']
prompt+= f"""
Dialogue:

{dialogue_to_summarize}

Summary:
"""

In [55]:
prompt

"\nDialogue:\n\n#Person1#: I have a problem with my cable.\n#Person2#: What about it?\n#Person1#: My cable has been out for the past week or so.\n#Person2#: The cable is down right now. I am very sorry.\n#Person1#: When will it be working again?\n#Person2#: It should be back on in the next couple of days.\n#Person1#: Do I still have to pay for the cable?\n#Person2#: We're going to give you a credit while the cable is down.\n#Person1#: So, I don't have to pay for it?\n#Person2#: No, not until your cable comes back on.\n#Person1#: Okay, thanks for everything.\n#Person2#: You're welcome, and I apologize for the inconvenience.\n\nSummary:\n#Person1# has a problem with the cable. #Person2# promises it should work again and #Person1# doesn't have to pay while it's down.\n\nDialogue:\n\n#Person1#: Would you like me to show you our new cleaning unit? It's a clever design.\n#Person2#: Yes, I'd like to see that. What does it clean exactly?\n#Person1#: It washes the solvent off all the metal part

In [57]:
#Prompt engineering - One shot inferencing
inputs = tokenizer(prompt, return_tensors='pt')
model_output = original_model.generate(inputs["input_ids"], max_new_tokens=50)[0]
output = tokenizer.decode(model_output,skip_special_tokens=True)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

INPUT PROMPT:

Dialogue:

#Person1#: I have a problem with my cable.
#Person2#: What about it?
#Person1#: My cable has been out for the past week or so.
#Person2#: The cable is down right now. I am very sorry.
#Person1#: When will it be working again?
#Person2#: It should be back on in the next couple of days.
#Person1#: Do I still have to pay for the cable?
#Person2#: We're going to give you a credit while the cable is down.
#Person1#: So, I don't have to pay for it?
#Person2#: No, not until your cable comes back on.
#Person1#: Okay, thanks for everything.
#Person2#: You're welcome, and I apologize for the inconvenience.

Summary:
#Person1# has a problem with the cable. #Person2# promises it should work again and #Person1# doesn't have to pay while it's down.

Dialogue:

#Person1#: Would you like me to show you our new cleaning unit? It's a clever design.
#Person2#: Yes, I'd like to see that. What does it clean exactly?
#Person1#: It washes the solvent off all the metal parts - the bl

In [62]:
#Prompt engineering - Few shot inferencing
example_indices_full = [40, 80, 120]
prompt = ''
for index in example_indices_full:
  dialogue = dataset['train'][index]['dialogue']
  summary = dataset['train'][index]['summary']
  prompt+=f"""
  Dialogue:

  {dialogue}

  Summary:

  {summary}
  """

example_index_to_summarize = 300
dialogue_to_summarize = dataset['train'][example_index_to_summarize]['dialogue']
summarized_dialogue = dataset['train'][example_index_to_summarize]['summary']
prompt+= f"""
Dialogue:

{dialogue_to_summarize}

Summary:
"""


In [65]:
#Prompt engineering - One shot inferencing
inputs = tokenizer(prompt, return_tensors='pt')
model_output = original_model.generate(inputs["input_ids"], max_new_tokens=50)[0]
output = tokenizer.decode(model_output,skip_special_tokens=True)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (1062 > 512). Running this sequence through the model will result in indexing errors


INPUT PROMPT:

  Dialogue:
  
  #Person1#: I just bought a new dress. What do you think of it?
#Person2#: You look really great in it. So are you going to a job interview or a party?
#Person1#: No, I was invited to give a talk in my school.
#Person2#: So how much did you pay for it?
#Person1#: I pay just $70 for it. I saved $30.
#Person2#: That's really a bargain.
#Person1#: You're right. Well, what did you do while I was out shopping?
#Person2#: I watched TV for a while and then I did some reading. It wasn't a very interesting book so I just read a few pages. Then I took a shower.
#Person1#: I thought you said you were going to see Mike.
#Person2#: I'll go and visit him at his home tomorrow. He'll return home tomorrow morning.
#Person1#: I'm glad he can finally returned home after that accident.
  
  Summary:
  
  While #Person1# made a bargain to buy a new dress, #Person2# watched TV, read a boring book, and took a shower at home.
  
  Dialogue:
  
  #Person1#: Hey, man. What's up?
#

LORA

In [6]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [23]:
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)
peft_model = get_peft_model(original_model,

                            lora_config)

In [24]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=40,
    logging_steps=1,
    max_steps=40
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets['validation']
)

In [25]:
peft_trainer.train()

peft_model_path="./peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,49.750000
2,46.000000
3,42.750000
4,37.750000
5,34.000000
6,30.375000
7,29.000000
8,26.750000
9,23.750000
10,21.875000


('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/spiece.model',
 './peft-dialogue-summary-checkpoint-local/added_tokens.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

In [26]:
index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids.cuda(), generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)


peft_model_outputs = peft_model.generate(input_ids=input_ids.cuda(), generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'DIALOGUE:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
DIALOGUE:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY

In [27]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids.cuda(), generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids.cuda(), generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,"@Ms.Dahm, I need to take a dictation for me.",The new policy will apply to all employees.
1,In order to prevent employees from wasting tim...,This memo is for internal and external communi...,This should go out as an intra-office memo to ...
2,Ms. Dawson takes a dictation for #Person1# abo...,This memo is intended to be a memo to all empl...,This memo is for the first day of the new offi...
3,#Person2# arrives late because of traffic jam....,@Person1#: I feel bad about how much my car is...,The EPA has a report on the pollution of the c...
4,#Person2# decides to follow #Person1#'s sugges...,The only problem is that I'm going to really m...,You're going to have to consider a new job.
5,#Person2# complains to #Person1# about the tra...,You're finally here!,You're finally here!
6,#Person1# tells Kate that Masha and Hero get d...,@@@@@@@@@@@@.,@Person1::
7,#Person1# tells Kate that Masha and Hero are g...,The rumors are that Masha and Hero are having ...,This conversation is about a couple who are ge...
8,#Person1# and Kate talk about the divorce betw...,The divorce is over.,@Meg_1:
9,#Person1# and Brian are at the birthday party ...,"You look great, you are absolutely glowing.",Today is Brian's birthday.


In [28]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)

print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.13816616042154062, 'rouge2': 0.042391304347826085, 'rougeL': 0.12545609493672621, 'rougeLsum': 0.12412757925526936}
PEFT MODEL:
{'rouge1': 0.1638112736838374, 'rouge2': 0.023469387755102038, 'rougeL': 0.1378750191220427, 'rougeLsum': 0.13609983748796695}
